In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn import tree

In [2]:
df = pd.read_csv('../Datasets/loan-prediction/train.csv')

In [3]:
print(df.head())

    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0   

In [4]:
df = df.dropna()
df = df.drop('Loan_ID', axis=1)
df = df.reset_index(drop=True)

In [5]:
print(df.head())

  Gender Married Dependents     Education Self_Employed  ApplicantIncome  \
0   Male     Yes          1      Graduate            No             4583   
1   Male     Yes          0      Graduate           Yes             3000   
2   Male     Yes          0  Not Graduate            No             2583   
3   Male      No          0      Graduate            No             6000   
4   Male     Yes          2      Graduate           Yes             5417   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
0             1508.0       128.0             360.0             1.0   
1                0.0        66.0             360.0             1.0   
2             2358.0       120.0             360.0             1.0   
3                0.0       141.0             360.0             1.0   
4             4196.0       267.0             360.0             1.0   

  Property_Area Loan_Status  
0         Rural           N  
1         Urban           Y  
2         Urban           Y  
3 

In [6]:
df['Dependents'].value_counts()

0     274
2      85
1      80
3+     41
Name: Dependents, dtype: int64

In [7]:
train, test = train_test_split(df, test_size=.2,)

In [8]:
train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
400,Male,Yes,0,Graduate,No,2130,6666.0,70.0,180.0,1.0,Semiurban,N
333,Female,No,1,Not Graduate,No,4606,0.0,81.0,360.0,1.0,Rural,N
195,Male,Yes,0,Not Graduate,No,1668,3890.0,201.0,360.0,0.0,Semiurban,N
173,Male,Yes,2,Graduate,No,4009,1717.0,116.0,360.0,1.0,Semiurban,Y
328,Male,Yes,0,Graduate,No,3246,1417.0,138.0,360.0,1.0,Semiurban,Y


In [9]:
test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
203,Male,Yes,0,Graduate,No,14683,2100.0,304.0,360.0,1.0,Rural,N
271,Male,No,0,Graduate,No,8750,4167.0,308.0,360.0,1.0,Rural,N
46,Male,Yes,2,Not Graduate,No,3357,2859.0,144.0,360.0,1.0,Urban,Y
366,Male,Yes,1,Graduate,Yes,3450,2079.0,162.0,360.0,1.0,Semiurban,Y
339,Male,Yes,0,Graduate,No,2425,2340.0,143.0,360.0,1.0,Semiurban,Y


In [10]:
X_train = train.drop('Loan_Status', axis=1)
X_train = pd.get_dummies(X_train)
Y_train = train['Loan_Status']
X_test = test.drop('Loan_Status', axis=1)
X_test = pd.get_dummies(X_test)
Y_test = test['Loan_Status']

In [11]:
X_train.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
400,2130,6666.0,70.0,180.0,1.0,0,1,0,1,1,0,0,0,1,0,1,0,0,1,0
333,4606,0.0,81.0,360.0,1.0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0
195,1668,3890.0,201.0,360.0,0.0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,0
173,4009,1717.0,116.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0
328,3246,1417.0,138.0,360.0,1.0,0,1,0,1,1,0,0,0,1,0,1,0,0,1,0


In [12]:
Y_train.head()

400    N
333    N
195    N
173    Y
328    Y
Name: Loan_Status, dtype: object

In [13]:
decision_tree = tree.DecisionTreeClassifier(random_state=1)

In [14]:
svc = SVC(
    kernel='poly',
)

In [15]:
bagging = ensemble.BaggingClassifier(
    base_estimator=decision_tree,
    n_estimators=10,
    random_state=0,
)

In [16]:
model = bagging.fit(X_train, Y_train)

In [17]:
model.score(X_test, Y_test)

0.7395833333333334

In [18]:
rf = ensemble.RandomForestClassifier(
    #criterion='gini',
)

In [19]:
model = rf.fit(X_train, Y_train)

In [20]:
model.score(X_test, Y_test)

0.7708333333333334

In [21]:
adaboost = ensemble.AdaBoostClassifier(
    #learning_rate=.01,
    #base_estimator=decision_tree,
    #algorithm='SAMME',
    random_state=1,
)

In [22]:
model = adaboost.fit(X_train, Y_train)

In [23]:
model.score(X_test, Y_test)

0.7395833333333334

In [24]:
gboost = ensemble.GradientBoostingClassifier(
    learning_rate=.01,
    random_state=1,
)

In [25]:
model = gboost.fit(X_train, Y_train)

In [26]:
model.score(X_test, Y_test)

0.75

In [27]:
xgbr = xgb.XGBClassifier(
    learning_rate=.01,
    random_state=1,
)

In [28]:
model = xgbr.fit(X_train, Y_train)

In [29]:
model.score(X_test, Y_test)

0.7291666666666666